<a href="https://colab.research.google.com/github/Udit0104/ML/blob/main/Binning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [3]:
df=pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(714, 3)

In [6]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


**Before Discretizer**

In [7]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
X_train.head(2)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542


In [10]:
clf=DecisionTreeClassifier()

In [11]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [12]:
accuracy_score(y_test,y_pred)

0.6293706293706294

In [13]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6288928012519561)

**Applying Dicretizer**

In [25]:
kbin_age=KBinsDiscretizer(n_bins=20,encode='ordinal',strategy='quantile')
kbin_fare=KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [26]:
trf=ColumnTransformer([
    ('first', kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [27]:
X_train_trf=trf.fit_transform(X_train)
X_test_trf=trf.transform(X_test)

In [28]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  4.  , 14.  , 18.  , 19.  , 21.  , 22.  , 24.  , 25.  ,
              27.  , 28.5 , 30.  , 32.  , 34.  , 36.  , 39.  , 42.  , 45.5 ,
              50.  , 57.5 , 80.  ])                                         ],
      dtype=object)

In [29]:
output=pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [30]:
output['age_labels']=pd.cut(x=X_train['Age'], bins=trf.named_transformers_['first'].bin_edges_[0], include_lowest=True)
output['fare_labels']=pd.cut(x=X_train['Fare'], bins=trf.named_transformers_['second'].bin_edges_[0], include_lowest=True)

In [31]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
769,32.00,12.0,8.3625,4.0,"(30.0, 32.0]","(8.158, 10.5]"
620,27.00,9.0,14.4542,7.0,"(25.0, 27.0]","(13.0, 14.454]"
305,0.92,0.0,151.5500,14.0,"(0.419, 4.0]","(108.9, 512.329]"
487,58.00,19.0,29.7000,10.0,"(57.5, 80.0]","(26.55, 31.275]"
514,24.00,7.0,7.4958,1.0,"(22.0, 24.0]","(7.25, 7.775]"


In [32]:
# Again Decision Tree on transformed data
clf=DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2=clf.predict(X_test_trf)

In [33]:
accuracy_score(y_test,y_pred2)

0.6433566433566433

In [34]:
X_trf=trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6289319248826292)